# Clase 01: Conceptos básicos de NLP

Instalar y cargar las librerías como sigue a continuación: 

In [34]:
# si no tiene alguna librería, ejecutar en un bloque lo siguiente:
# !pip install -U nombre_libreria

import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
from tqdm.auto import tqdm
import csv
import os
import emoji

# Dataset

Usaremos un dataset de mensajes de odio en español alojado en <https://zenodo.org/record/2592149#.Ystdn3bMKUl>

In [13]:
cwd = os.getcwd()
print(os.path.join(cwd, "labeled_corpus_6K.txt"))

d:\dev\MTI-455\notebooks\labeled_corpus_6K.txt


In [37]:
df = pd.read_csv(os.path.join(cwd, "labeled_corpus_6K.txt"),
                         sep = "\t", 
                         encoding = "utf8",
                         quotechar = "\"",
                         names=['id', 'text', 'class'], 
                         header=None) 

In [27]:
df.head(5)

,id,text,class
0,828025263321657348,Ismael es egocentrico porque se vuelve loca si...,0
1,828025128797741057,..ya tardaba en salir quien pronunciase nombre...,0
2,828025087815274496,"(Esto no es un discurso político y razonado, o...",0
3,828025006626058241,"Muy despreciados,siiii,pero todos vestidos de ...",1
4,828024709761658880,marica explicame porque a veces no te entiendo...,1


# Pre-procesamos el texto

Utilizaremos un archivo llamado ``preprocessing_tweets.py`` que contiene diversas funciones para limpiar el texto.



In [42]:
# todo a minúscula.
from preprocessing_tweets import *
df['text_prepo'] = df['text'].str.lower()

In [39]:
df.head()

,id,text,class,text_prepo
0,828025263321657348,Ismael es egocentrico porque se vuelve loca si...,0,ismael es egocentrico porque se vuelve loca si...
1,828025128797741057,..ya tardaba en salir quien pronunciase nombre...,0,..ya tardaba en salir quien pronunciase nombre...
2,828025087815274496,"(Esto no es un discurso político y razonado, o...",0,"(esto no es un discurso político y razonado, o..."
3,828025006626058241,"Muy despreciados,siiii,pero todos vestidos de ...",1,"muy despreciados,siiii,pero todos vestidos de ..."
4,828024709761658880,marica explicame porque a veces no te entiendo...,1,marica explicame porque a veces no te entiendo...


Eliminamos menciones de URL

In [43]:
# eliminamos urls
tqdm.pandas()
df['text_prepo'] = df['text_prepo'].progress_apply(lambda text: remove_mentions_urls(text))

100%|██████████| 5938/5938 [00:00<00:00, 247353.51it/s]


In [45]:
# eliminamos hashtags
df['text_prepo'] = df['text_prepo'].progress_apply(lambda text: remove_hts(text))

100%|██████████| 5938/5938 [00:00<00:00, 269899.40it/s]


In [46]:
# eliminamos puntuacion
df['text_prepo'] = df['text_prepo'].progress_apply(lambda text: remove_punctuation(text))

100%|██████████| 5938/5938 [00:00<00:00, 98908.99it/s]


In [47]:
# eliminamos numeros
df['text_prepo'] = df['text_prepo'].progress_apply(lambda text: remove_numbers(text))

100%|██████████| 5938/5938 [00:00<00:00, 247695.45it/s]


In [48]:
# eliminamos tildes
df['text_prepo'] = df['text_prepo'].progress_apply(lambda text: remove_accents(text))

100%|██████████| 5938/5938 [00:00<00:00, 588928.28it/s]


In [49]:
# eliminamos stopwords
stopwords = create_stopwords()
df['text_prepo'] = df['text_prepo'].progress_apply(lambda text: remove_stopwords(text, stopwords))

100%|██████████| 5938/5938 [00:00<00:00, 258187.27it/s]


In [51]:
df['text_prepo'] = df['text_prepo'].progress_apply(lambda text: remove_dots(text))

100%|██████████| 5938/5938 [00:00<00:00, 592768.88it/s]


In [52]:
df.head()

,id,text,class,text_prepo
0,828025263321657348,Ismael es egocentrico porque se vuelve loca si...,0,ismael egocentrico vuelve loca pelo bonito😂😂😂😂...
1,828025128797741057,..ya tardaba en salir quien pronunciase nombre...,0,tardaba salir pronunciase nombre catalan silab...
2,828025087815274496,"(Esto no es un discurso político y razonado, o...",0,discurso politico razonado obviamente llamada ...
3,828025006626058241,"Muy despreciados,siiii,pero todos vestidos de ...",1,despreciados siiii vestidos alta costura españ...
4,828024709761658880,marica explicame porque a veces no te entiendo...,1,marica explicame entiendo — venir ensuciar cc ...
